-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [1]:
#import libraries
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [5]:
def extract_features(filename):
    y, sr = librosa.load(filename)
    features = {}
    features['mfcc'] = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).flatten()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr).flatten()
    features['mel'] = librosa.feature.melspectrogram(y=y, sr=sr).flatten()
    return features
    # pass

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [36]:
def load_data(test_size):
    # directories = ["Audio files Song", "Audio files Speech"]
    directories = [os.path.join(os.getcwd(), "Audio files Song"), os.path.join(os.getcwd(), "Audio files Speech")]
    for directory in directories:
        subdirectories = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
    for subdirectory in subdirectories:
        files = [os.path.join(directory, subdirectory, name) for name in os.listdir(os.path.join(directory, subdirectory))]
    X = []
    y = []
    for file in files:
        features = extract_features(file)
        filename = os.path.basename(file)
        emotion_code = int(filename.split("-")[2])
        emotion_one_hot = np.zeros(8)
        emotion_one_hot[emotion_code - 1] = 1      
        X.append(features)
        y.append(emotion_one_hot)
    
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, X_test, y_train, y_test
    # pass

Use the load_data function to load data and split the data using the train_test_split function.

In [37]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, y_train, y_test = load_data(0.2)

Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [8]:
#define the model.
model = None

Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [ ]:
#fit the model on training data with validation split of 0.2

In [ ]:
#predict for testing data and printout the accuracy

Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [ ]:
#print the classification report for the model
from sklearn.metrics import classification_report
# report = classification_report(y_test, y_pred)
# print(report)

Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)